# Twitter Pre-processing

Twitter Pre-processing to build following:

1. Twitter Timeline Data
2. Twitter User Profile Data
3. Twitter Most Frequent Words Data
4. Twitter Most Frequent Hashtags Data
5. Twitter Most Frequent Mentions Data

In [4]:
import json
import requests
from requests_oauthlib import OAuth1
import json
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import tweepy
import seaborn as sns
%matplotlib inline
import os
import pandas as pd
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn')

In [317]:
campus = "iupui"
twitter_file = "iupui"

## Settings

In [318]:
raw_data_dir = f"../../data/raw_data/{campus}/twitter/"
raw_file_data = f"{twitter_file}_twitter_data.csv"
raw_file_userinfo = f"{twitter_file}_twitter_user_profile_data.csv"


processed_data_dir =  f"../../data/processed_data/{campus}/twitter/"
file = f"{twitter_file}_twitter_data.xlsx"


####

### twitter data - main sheet
twitter_data_sheetname = "twitter_data"

## user profile sheet 
twitter_user_profile_data_sheetname = "user_profile_data"

## Most frequent words
most_frequent_words_sheetname = "most_freq_100_words"

## Most frequent hashtags
most_frequent_hashtags_sheetname = "most_freq_100_hashtags"

## Most frequent screennames
most_frequent_screennames_sheetname = "most_freq_100_mentions"

###



In [319]:
import nltk
stop_words =   nltk.corpus.stopwords.words('english') + [
 '.',
 ',',
 '--',
 '\'s',
 '?',
 ')',
 '(',
 ':',
 '\'',
 '\'re',
 '"',
 '-',
 '}',
 '{',
 u'—',
     
 ]

In [320]:
## Create Directory if it doesnt exists
def make_dir_if_not_exists(directory):
    dir_path = Path(directory)
    if not dir_path.exists():
        os.makedirs(dir_path)
    else:
        print("Path exists")
    
make_dir_if_not_exists(processed_data_dir)
    

Path exists


## Read Data

In [321]:
import shutil  

## raw file
#shutil.copyfile(os.path.join(raw_data_dir,raw_file_data), os.path.join(processed_data_dir,raw_file_data))
#print("Copied data file!")
#shutil.copyfile(os.path.join(raw_data_dir,raw_file_userinfo), os.path.join(processed_data_dir,raw_file_userinfo))
#print("Copied user file!")



In [322]:
## read data
import csv
df = pd.read_csv(os.path.join(raw_data_dir,raw_file_data))
user_data_df = pd.read_csv(os.path.join(raw_data_dir,raw_file_userinfo))

## save currentfile 
#df.to_excel(os.path.join(processed_data_dir,"indiana-university-bl_twitter_data.xls"),index=False)

df["created"] = pd.to_datetime(df["created"])

In [323]:
print("No of rows and columns: {}, min date: {}, max date: {}".format(df.shape,df['created'].min(),df['created'].max()))
      

No of rows and columns: (2374, 22), min date: 2018-06-08 12:59:00, max date: 2019-06-13 19:43:00


In [324]:
df.head()

,created,text,polarity,subjectivity,tweet_id,truncated,user,user_followers_count,user_friends_count,user_favourites_count,...,retweet_count,favorite_count,reply_count,at_mentions,hashtags,urls,source,lat,long,tweet_type
0,2019-06-13 19:43:00,"@Ryansully11 @IUPUIBookstore When online orders are processed, the bookstore sends an auto email to the address you… https://t.co/yF9lU0DwiT",0.000000,0.000000,1.139260e+18,True,IUPUI,35744,3154,4936,...,0,0,0,"Ryansully11, IUPUIBookstore",NaN,https://t.co/yF9lU0DwiT,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,tweet
1,2019-06-13 19:02:00,😍😍 https://t.co/kv67FaZoPC,0.000000,0.000000,1.139250e+18,False,IUPUI,35744,3154,4936,...,0,19,0,NaN,NaN,https://t.co/kv67FaZoPC,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,quote
2,2019-06-13 18:45:00,"@Ryansully11 Hi, Ryan: We spoke to the @IUPUIBookstore last time you tweeted about this, and they assured that your… https://t.co/rfcKFxta9X",0.000000,0.066667,1.139240e+18,True,IUPUI,35744,3154,4936,...,0,0,0,"Ryansully11, IUPUIBookstore",NaN,https://t.co/rfcKFxta9X,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,tweet
3,2019-06-13 18:00:00,#tbt Recognize any familiar spots? 🔎 A map of our campus as it looked in 1969.\n.\n#MyIUPUI History:… https://t.co/AOgi2mYSwY,0.375000,0.500000,1.139230e+18,True,IUPUI,35744,3154,4936,...,3,9,0,NaN,"tbt, MyIUPUI",https://t.co/AOgi2mYSwY,"<a href=""https://sproutsocial.com"" rel=""nofollow"">Sprout Social</a>",NaN,NaN,tweet
4,2019-06-13 17:06:00,"RT @paydar: A great reason to stay for summer session, Jags --&gt; getting to see how beautiful our campus is on a cool June day. https://t.co…",0.666667,0.800000,1.139220e+18,False,IUPUI,35744,3154,4936,...,2,0,0,paydar,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",NaN,NaN,retweet


## Twitter Sentiment - Words

In [325]:
import re
import string

from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

import nltk


from nltk.corpus import stopwords 
stopwords_english = stopwords.words('english')

from nltk.tokenize import TweetTokenizer 
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

nltk.download("movie_reviews")
nltk.download('punkt')


class TweetSentiment:
    def __init__(self):

        self.analysis = ""
        
     

    @staticmethod
    
    def clean_tweet(tweet_text):
        """
        1. remove retweet "RT"
        2. remove hyperlinks
        3. remove hashtags 
        4. remove stopwords
        5. remove emoticons 
        6. remove punctuation, full-stop and comma, exclamation sign etc.
        7. convert words to stem/base words using porter stemming algorithm
        """
        
        # Happy Emoticons
        emoticons_happy = set([':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
                                    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
                                    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
                                    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
                                    '<3'])
        # Sad Emoticons
        emoticons_sad = set([':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
                                  ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
                                  ':c', ':{', '>:\\', ';('])
        # all emoticons (happy + sad)
        emoticons =  emoticons_happy.union(emoticons_sad)
        
        
        
        ## remove "RT"
        tweet = re.sub(r'^RT[\s]+','',tweet_text)
        
        ## remove hyperlinks
        tweet = re.sub(r'https?:\/\/.*[\r\n]*','',tweet)
        
        ## remove hashtags
        tweet = re.sub(r'#','',tweet)
        
        ## tokenize tweets
        # tokenize tweets
        tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
        tweet_tokens = tokenizer.tokenize(tweet)
        
        tweets_clean = []    
        for word in tweet_tokens:
            if (word not in stopwords_english and # remove stopwords
                word not in emoticons  and # remove emoticons
                word not in string.punctuation): # remove punctuation
                stem_word = stemmer.stem(word) # stemming word
                tweets_clean.append(stem_word)
                
        return " ".join(tweets_clean)
        
        
        
    @staticmethod
    def remove_punctuation(tweet_text):
        return tweet_text.str.replace('[^\w\s]','')
    """
    Get sentiment of the tweets
    """
    def get_tweet_sentiment(self,tweet):
        self.analysis = TextBlob(TweetSentiment.remove_punctuation(TweetSentiment.clean_tweet(tweet)))

        if self.analysis.sentiment.polarity>0:
            return "positive"
        elif self.analysis.sentiment.polarity==0:
            return "neutral"
        else:
            return "negative"


    """
    Get sentiment data
    """
    def get_tweet_sentiment_data(self,tweet):
        
        analysis = TextBlob(TweetSentiment.clean_tweet(tweet))
        
        #classification= analysis.sentiment.classification
        #positive      = analysis.sentiment.p_pos
        #negative      = analysis.sentiment.p_neg
        polarity  = analysis.polarity
        subjectivity = analysis.subjectivity
        if polarity < 0:
            sentiment = "Negative"
        elif polarity > 0:
            sentiment = "Positive"
        else:
            sentiment = "Neutral"
        
        return  sentiment, analysis.words,polarity
    
    
#df["text"] = df['text'].apply(lambda x: str(TextBlob(x).correct())) 

sentiment_obj = TweetSentiment()
wordlist = []
sentiment_list = []
polarity_list = []
for index, row in df.iterrows():
    
    sentiment, words,polarity = sentiment_obj.get_tweet_sentiment_data(str(row["text"]))
    row["words"] = ", ".join(words)
    row["sentiment"] = sentiment
    polarity_list.append(polarity)
    wordlist.append(row["words"])
    sentiment_list.append(sentiment)
    



[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/mchivuku/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mchivuku/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [326]:
df["words"]= pd.Series(wordlist)
df["sentiment"]= pd.Series(sentiment_list)
df['polarity'] = pd.Series(polarity_list)

In [327]:
pd.set_option('display.max_colwidth',-1)

df[df['polarity']<0].shape


(167, 24)

## Most Frequent Words


In [347]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import PorterStemmer

def tweet_tokenizer(txt):
    try:
        tokenizer = TweetTokenizer()
        all_tokens = tokenizer.tokenize(txt.lower())
        # this line filters out all tokens that are entirely non-alphabetic characters
        filtered_tokens = [t for t in all_tokens if t.islower()]
        # filter out all tokens that are <=2 chars
        filtered_tokens = [x for x in filtered_tokens if len(x)>2]
    except IndexError:
        filtered_tokens = []
    return(filtered_tokens)


tweets_text = str(df["text"])

In [348]:
tweet_tokenizer(tweets_text)

['@ryansully11',
 '@iupuibookstore',
 'when',
 'online',
 'orders',
 'are',
 'processed',
 'the',
 'bookstore',
 'sends',
 'auto',
 'email',
 'the',
 'address',
 'you',
 'https://t.co/yf9lu0dwit',
 'https://t.co/kv67fazopc',
 '@ryansully11',
 'ryan',
 'spoke',
 'the',
 '@iupuibookstore',
 'last',
 'time',
 'you',
 'tweeted',
 'about',
 'this',
 'and',
 'they',
 'assured',
 'that',
 'your',
 'https://t.co/rfckfxta9x',
 '#tbt',
 'recognize',
 'any',
 'familiar',
 'spots',
 'map',
 'our',
 'campus',
 'looked',
 '#myiupui',
 'history',
 'https://t.co/aogi2myswy',
 '@paydar',
 'great',
 'reason',
 'stay',
 'for',
 'summer',
 'session',
 'jags',
 'getting',
 'see',
 'how',
 'beautiful',
 'our',
 'campus',
 'cool',
 'june',
 'day',
 'https://t.co…',
 '@iupuijaguars',
 'happy',
 '#worldsoftballday',
 'from',
 'your',
 '@iupuisoftball',
 'squad',
 '#hlsb',
 '#jagsroar',
 'https://t.co/h5gatisshc',
 '@innovateindiana',
 'lessons',
 'learned',
 'young',
 '#startup',
 'founder',
 'part',
 'custome

In [349]:
## Get most frequent words
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

def get_most_frequent_words(text_series, stop_words = None, ngram_range = (1,2)):
    
    count_vectorizer = CountVectorizer(analyzer = "word",
                                       tokenizer = tweet_tokenizer,
                                       stop_words = stop_words,
                                       ngram_range = ngram_range 
                                      )
    
    term_freq_matrix = count_vectorizer.fit_transform(text_series)
        
    terms = count_vectorizer.get_feature_names()
    term_frequencies = term_freq_matrix.sum(axis = 0).tolist()[0]
    
    
    term_freq_df = (pd.DataFrame(list(zip(terms, term_frequencies)), columns = ["token","count"])
                    .set_index("token")
                    .sort_values("count",ascending = False))
    
    ### Avoiding hashtags and mentions
    data = term_freq_df.index.str.startswith(("@","#"))

    data = [not(data[i]) for i in range(len(data))]
    term_freq_df = term_freq_df[data]
    return term_freq_df

term_freq_df = get_most_frequent_words(df["text"],stop_words = stop_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mchivuku/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [350]:
term_freq_df = term_freq_df[:100]

## Most Frequent Hashtags and Screennames

In [351]:
df.columns

Index(['created', 'text', 'polarity', 'subjectivity', 'tweet_id', 'truncated',
       'user', 'user_followers_count', 'user_friends_count',
       'user_favourites_count', 'user_statuses_count', 'retweeted_status',
       'retweet_count', 'favorite_count', 'reply_count', 'at_mentions',
       'hashtags', 'urls', 'source', 'lat', 'long', 'tweet_type', 'words',
       'sentiment', 'created date'],
      dtype='object')

In [352]:
hashtags = []
screen_names = []
for index, row in df.iterrows():
    if isinstance(row["hashtags"], str):
        hashtags.extend([ t for t in str(row["hashtags"]).split(", ")])
    if isinstance(row["at_mentions"], str):
        screen_names.extend([t for t in str(row["at_mentions"]).split(", ")]) 
    


In [353]:
print("Hashtags: ", len(hashtags),"Screen_names:", len(screen_names))


Hashtags:  588 Screen_names: 2386


#### Build Hashtag and Mentions table

In [354]:
from prettytable import PrettyTable
from collections import Counter
for label, data in (('Screen Name', screen_names),
                    ('Hashtag', hashtags)):
    pt = PrettyTable(field_names=[label, 'Count'])
    
    
    ### Merge duplicates
    c = {}
   
    for k in list(data):
        keys = [str.lower(p) for p in c.keys()]
        if str.lower(k) in keys:
            current_keys = list(c.keys())
            match_index = keys.index(str.lower(k))
            match = current_keys[match_index]
            c[match] += 1
        else:
            c[k] = 1
    
    if label == 'Screen Name':
        screenname_df = pd.DataFrame.from_dict(c, orient='index').reset_index()
        screenname_df = screenname_df.rename(columns={'index':'screen_name', 0:'count'}).sort_values("count",ascending = False)
        print(screenname_df.head())
    else:
        hashtag_df = pd.DataFrame.from_dict(c, orient='index').reset_index()
        hashtag_df = hashtag_df.rename(columns= {'index':"hashtag",0:'count'}).sort_values("count",ascending = False)
        print(hashtag_df.head())
    

     screen_name  count
11  IUPUI         373  
2   paydar        225  
3   IUPUIJaguars  59   
12  IndianaUniv   54   
81  iupuiSAPB     42   
        hashtag  count
1   MyIUPUI      43   
4   JagsROAR     39   
9   iupui        27   
12  Indy         20   
23  iupuigrad19  20   


In [355]:
hashtag_df.head()

,hashtag,count
1,MyIUPUI,43
4,JagsROAR,39
9,iupui,27
12,Indy,20
23,iupuigrad19,20


In [356]:
print(f"Screennames: {screenname_df.shape}, hashtags: {hashtag_df.shape}")

Screennames: (634, 2), hashtags: (223, 2)


### Top 100 - mentions and hashtags

In [357]:
screenname_df, hashtag_df  = screenname_df[:100], hashtag_df[:100]

### Add Created Date


In [358]:
df['created date'] = df["created"].dt.date


## Build Excel Sheets

In [359]:
sentiment_df = df.loc[:,['sentiment']]


In [360]:
#df = df.drop(['Created Date'],axis=1)
df.head()

,created,text,polarity,subjectivity,tweet_id,truncated,user,user_followers_count,user_friends_count,user_favourites_count,...,at_mentions,hashtags,urls,source,lat,long,tweet_type,words,sentiment,created date
0,2019-06-13 19:43:00,"@Ryansully11 @IUPUIBookstore When online orders are processed, the bookstore sends an auto email to the address you… https://t.co/yF9lU0DwiT",0.000,0.000000,1.139260e+18,True,IUPUI,35744,3154,4936,...,"Ryansully11, IUPUIBookstore",NaN,https://t.co/yF9lU0DwiT,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,tweet,"onlin, order, process, bookstor, send, auto, email, address, …",Neutral,2019-06-13
1,2019-06-13 19:02:00,😍😍 https://t.co/kv67FaZoPC,0.000,0.000000,1.139250e+18,False,IUPUI,35744,3154,4936,...,NaN,NaN,https://t.co/kv67FaZoPC,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,quote,"😍, 😍",Neutral,2019-06-13
2,2019-06-13 18:45:00,"@Ryansully11 Hi, Ryan: We spoke to the @IUPUIBookstore last time you tweeted about this, and they assured that your… https://t.co/rfcKFxta9X",0.000,0.066667,1.139240e+18,True,IUPUI,35744,3154,4936,...,"Ryansully11, IUPUIBookstore",NaN,https://t.co/rfcKFxta9X,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,tweet,"hi, ryan, spoke, last, time, tweet, assur, …",Neutral,2019-06-13
3,2019-06-13 18:00:00,#tbt Recognize any familiar spots? 🔎 A map of our campus as it looked in 1969.\n.\n#MyIUPUI History:… https://t.co/AOgi2mYSwY,0.375,0.500000,1.139230e+18,True,IUPUI,35744,3154,4936,...,NaN,"tbt, MyIUPUI",https://t.co/AOgi2mYSwY,"<a href=""https://sproutsocial.com"" rel=""nofollow"">Sprout Social</a>",NaN,NaN,tweet,"tbt, recogn, familiar, spot, 🔎, map, campu, look, 1969, myiupui, histori, …",Positive,2019-06-13
4,2019-06-13 17:06:00,"RT @paydar: A great reason to stay for summer session, Jags --&gt; getting to see how beautiful our campus is on a cool June day. https://t.co…",0.575,0.800000,1.139220e+18,False,IUPUI,35744,3154,4936,...,paydar,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",NaN,NaN,retweet,"great, reason, stay, summer, session, jag, get, see, beauti, campu, cool, june, day",Positive,2019-06-13


<ul>
<li> Twitter Data </li>
<li> User Profile Data </li>
<li> Most Frequent Words </li>
<li> Most Frequent Hashtags </li>
<li> Most Frequent Mentions </li>

</ul>

In [361]:
user_data_df

,user_id,name,screen_name,followers_count,friends_count,favourites_count,statuses_count,total_reach,avg_follower_cnt
0,10086382,IUPUI,IUPUI,35744,3154,4936,26856,12744495,356.55


In [362]:

with pd.ExcelWriter(os.path.join(processed_data_dir,file),engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name=twitter_data_sheetname,index=False)
    user_data_df.to_excel(writer, sheet_name=twitter_user_profile_data_sheetname,index=False)
    term_freq_df.to_excel(writer, sheet_name=most_frequent_words_sheetname)
    screenname_df.to_excel(writer, sheet_name=most_frequent_screennames_sheetname,index=False)
    hashtag_df.to_excel(writer, sheet_name=most_frequent_hashtags_sheetname,index=False)
    writer.save()
    